In [ ]:
#### merging datasets from pilot0 (N=20) and pilot1 (N=81)

In [82]:
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import tabular as tb
import pandas as pd
import json
import re

from PIL import Image
import base64

from helpers import *
from analysis_helpers import *
import analysis_helpers as h
reload(h)

<module 'analysis_helpers' from 'analysis_helpers.py'>

In [115]:
# directory & file hierarchy
exp_path = '3dObjects/sketchpad_basic'
analysis_dir = os.getcwd()
data_dir = os.path.abspath(os.path.join(os.getcwd(),'../../..','data',exp_path))
exp_dir = os.path.abspath(os.path.join(os.getcwd(),'../../..','experiments',exp_path))
sketch_dir = os.path.abspath(os.path.join(os.getcwd(),'../../..','analysis',exp_path,'sketches','pilot1'))

In [116]:
D0 = pd.read_csv(os.path.join(os.path.join(analysis_dir,'sketchpad_basic_pilot0_group_data.csv')))
D1 = pd.read_csv(os.path.join(os.path.join(analysis_dir,'sketchpad_basic_pilot1_group_data.csv')))

In [117]:
_D = pd.concat([D0,D1], join="inner",ignore_index=True)

In [118]:
## get list of all valid gameids across session
_valid_p0 = np.unique(D[D['iteration']=='pilot0']['gameID'].values)

## 10/1/17: discussion with @rxdh: flag/exclude a game where the sketcher annotated drawing with words on at least two of the trials
wordy = ['4196-e2f9bf3f-e1cf-4e54-8188-f5f3d07f171b']
valid_p0 = [i for i in _valid_p0 if i not in wordy]
tmp = pd.read_csv('valid_gameids_pilot1.csv')
valid_p1 = np.unique(tmp['valid_gameids'].values)
valid_gameids = list(valid_p0) + list(valid_p1)
print '# valid games: ' + str(len(valid_gameids))

# valid games: 100


In [119]:
# filter so that only valid gameid's are included in the final merged group data csv, then save out
D = _D[_D['gameID'].isin(valid_gameids)]
D.to_csv('sketchpad_basic_merged_group_data.csv')

### Print out all sketches from both pilot0 and pilot1 in same way


In [83]:
## print out sketches with target & distractors from this game in separate folders
import traceback
backup_path_images = '/Users/judithfan/Dropbox/stimuli_repository/subordinate_allrotations_6_minified'

_valid_gameids = valid_gameids[:3]

target_sketch_dir = os.path.join(analysis_dir,'sketches','sketch')
target_3D_dir = os.path.join(analysis_dir,'sketches','target')
distractor1_3D_dir = os.path.join(analysis_dir,'sketches','distractor1')
distractor2_3D_dir = os.path.join(analysis_dir,'sketches','distractor2')
distractor3_3D_dir = os.path.join(analysis_dir,'sketches','distractor3')
out_paths = [target_3D_dir,distractor1_3D_dir,distractor2_3D_dir,distractor3_3D_dir]


if not os.path.exists(target_3D_dir):
    os.makedirs(os.path.join(analysis_dir,'sketches','target'))
    os.makedirs(os.path.join(analysis_dir,'sketches','distractor1'))
    os.makedirs(os.path.join(analysis_dir,'sketches','distractor2'))
    os.makedirs(os.path.join(analysis_dir,'sketches','distractor3'))
    os.makedirs(os.path.join(analysis_dir,'sketches','sketch'))    

run_this = 1
if run_this:
    for g in valid_gameids:
        print 'Printing out sketches from game: ' + g
        _D = D[(D.gameID==g)]
        _D = _D.sort_values(by=['target'])
        _i = 1
        textsize=12
        fig = plt.figure(figsize=(16,6))        
        for i,_d in _D.iterrows():
            imgData = _d['png']
            filestr = base64.b64decode(imgData)
            if not os.path.exists(target_sketch_dir):
                os.makedirs(target_sketch_dir)
            fname = os.path.join(target_sketch_dir,'gameID_' + _d['gameID'] + '_trial_' + str(_d['trialNum']) + '.png')
            with open(fname, "wb") as fh:
                fh.write(imgData.decode('base64'))
            im = Image.open(fname)
            im.save(fname)

            targetname = _d['target']
            distractors = [_d['Distractor1'],_d['Distractor2'],_d['Distractor3']]
            full_list = [_d['target'],_d['Distractor1'],_d['Distractor2'],_d['Distractor3']]         
            
            for (i,d) in enumerate(full_list):
                if os.path.exists(backup_path_images):
                    if not np.isnan(_d['pose']):
                        pose = _d['pose']
                        fn = os.path.join(backup_path_images,get_actual_pose(d,pose)[0])
                    else:
                        fn = os.path.join(backup_path_images,get_actual_pose(d,35)[0])
                else:
                    if hasattr(_d, 'pose'):
                        pose = _d['pose']
                        URL = build_url_from_filenames(get_actual_pose(d,pose)[0])
                    else:
                        URL = full_dict[d]
                    fn = cStringIO.StringIO(urllib.urlopen(URL).read())
                fig = plt.figure(figsize=(8,8))                    
                im = Image.open(fn)
                im = im.resize((256,256), Image.ANTIALIAS).convert('RGB')                                
                out_dirs = [target_3D_dir,distractor1_3D_dir,distractor2_3D_dir,distractor3_3D_dir]                
                out_path = os.path.join(out_dirs[i],'gameID_' + _d['gameID'] + '_trial_' + str(_d['trialNum']) + '.png')
                im.save(out_path)


Printing out sketches from game: 0497-f1cfe6e2-408b-4186-9faa-fb3a5132c84d
Printing out sketches from game: 0814-04fc8a68-3d7e-4233-895f-df3216084caa
Printing out sketches from game: 0908-40a14e14-9e59-43de-ae50-ad55ac4cd6c5
Printing out sketches from game: 1111-56f831fd-803f-410f-b69c-bc7dbdcc329f
Printing out sketches from game: 1141-ca308fef-84d5-4e97-86dd-62bddaac5077
Printing out sketches from game: 1959-9c408981-712c-4a3e-91d3-59d4c6fa60b1
Printing out sketches from game: 2237-202f824c-40fa-4470-97db-ad78bee58f5c
Printing out sketches from game: 3085-aac45f56-fcba-4cd3-8e58-05f8f61a0a62
Printing out sketches from game: 3175-9d2c7a95-0efd-49a5-aa90-91a036f1f684
Printing out sketches from game: 4065-893e6d81-fff6-4c5d-aaa5-d86d3fe6cdcb
Printing out sketches from game: 4196-e2f9bf3f-e1cf-4e54-8188-f5f3d07f171b
Printing out sketches from game: 4747-d7eba975-9806-4249-84fb-9c3c5235ed66
Printing out sketches from game: 5321-e1e2cfdf-17c1-414c-8c67-41aa15695acd
Printing out sketches fro

/Users/judithfan/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
### only consider correct trials